## Introduction à la reconnaissance faciale

<img src = "https://www.skillsuccess.com/wp-content/uploads/2018/01/python-image-recognition-2-460x307.jpg" width = "600">

Dans ce notebook, nous verrons comment il est possible de créer un algorithme de reconnaissance facial en python.
Pour ce faire, nous utiliserons les librairies **face_recognition**, **cv2** et **numpy**

Notre plan est organiser de la manière suivante:

1. Importation des librairies
2. Chargement et encodages des images
3. Détection d'image sur la webcam

### I - Importation des librairies

On commencera par importer les librairies utiles pour notre projet. Nous utiliserons particulièrement **face_recognition**.

In [2]:
############################################################################################
#####                   Algorithme de reconnaissance faciale
#############################################################################################
##### Imporation des librairies
############################################################################################
import face_recognition
import cv2
import numpy as np

#### Lancement de la webcam
video_capture = cv2.VideoCapture(0)


On définie ensuite une fonction de chargement des images qui prendra le fichier image en entrée et donnera en sortie un tableau de pixel (**image**) et un encodage de ce tableau (**image_encod**).

In [3]:
##############################################################################################
#### Fonction de chargement de l'image à reconnaitre 
##############################################################################################

def chargement_image(fichier):
    """
        Entrée:
            Le fichier image
            
        Sortie:
            L'image et son encodage
    """
    #### On initialise les tableaux pour les objets "image" et "image_encod"
    image = np.array([])
    image_encod = np.array([])
    image = face_recognition.load_image_file(fichier)
    image_encod = face_recognition.face_encodings(image)[0]
    return image, image_encod #### Sortie: Image et Encodage de l'image

### II - Chargement et encodages des images

Dans cette article, nous utiliserons les images les plus connues en France pour notre algorithme de reconnaissance. La première correspond à celle du **Président Macron**, la seconde est celle du **Premier Ministre Philippe**.
Evidemment, ne pouvant pas tester mon algorithme sur ces images, j'y rajouterais modestement la mienne à coté.
L'idée de ces deux premiers ajouts étant de **montrer que si l'on dispose de suffisament d'image, il sera possible de détecter celles-ci depuis notre caméra**.

<table>
    <td><img src = "E_Macron.png" width = "200px" ></td>
    <td><img src = "E_Philippe.png" width = "200px" ></td>
    <td><img src = "Moi.png" width = "200px" ></td>
</table>

On encode les 3 images précedentes pour les rendre compatible avec l'algorithme.

In [4]:
#### On crée trois nouvelles variables correspondant aux trois visages que l'on souhaite tester.

macron_img, macron_encod = chargement_image("E_macron.png")
philippe_img, philippe_encod = chargement_image("E_Philippe.png")
moi_img, moi_encod = chargement_image("moi.png")

On crée une liste de ces trois encodages ainsi qu'une liste des trois nom associée.

In [5]:
####################################################################################################
#### Création de liste d'encoding et de label
####################################################################################################

visage_connu_encoding = [
    moi_encod, 
    macron_encod,
    philippe_encod
]

nom_visage = [ "Yacine Aslimi",
                "Emmanuel Macron",
                "Edouard Philippe"
             ]

### III - Détection d'image sur la webcam

Une fois les images encodées, il nous reste à les comparer avec le flux de notre webcam. Le script se decompose de la manière suivante:

* Lancement de la webcam
* Détection des visages sur les images de la webcam
* Comparasion de chacune des images de la webcam avec les images prédéfinies
* Affichage du résultat

In [6]:
#####################################################################################################
#####                       Comparaison des images avec le flux de la webcam
#####################################################################################################

def activation_reconnaissance_image():
    """
    But: Lancer la webcam et identifier si l'image correspond à une des images définies précédement
        Entrée:
            Aucune
        Sortie:
            Video et filtre sur le visage
    """
    
    while True:
       
        # On lance la webcam , le ret correspond et le frame correspond à l'image 
        ret, frame = video_capture.read()
        
        #### On convertie les images BGR en RGB
        rgb_frame = frame[:, :, ::-1]

        
        #### Détection de tout les visages sur le frame définie à l'aide de la fonction face_location
        face_locations = face_recognition.face_locations(rgb_frame)
        #### On encode ces visages
        face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

         #### On boucle sur chacun des frames de la video
        
        for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):

            #### Comparaison entre le visage connu et le frame  
            matches = face_recognition.compare_faces(visage_connu_encoding, face_encoding)#### False, False, False...True...False
            """ Dans cette fonction, on compare les visages encodées issues de nos images avec les visages de chacuns des frames issues de notre video"""
            name = "Inconnu"

            #### Si il ya correspondance entre visage_connu_encoding et les visages issues su frame (visages_encod)

            if True in matches:###( liste de true false (ici 3) et on demande s'il ya un true, on prend son index et on applique à la liste des nom de visage pour avoir le nom de l'image)
                first_match_index = matches.index(True) ### Numero de l'index (présence dans la liste)
                name = nom_visage[first_match_index] #### Application à la liste des nom de visage (index 3)

            # Déssine un rectangle sur le frame ou l'image est présente
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)

            # Rédaction d'un texte sous le cadre
            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 255, 0), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)


        #### Affiche le résultat de la video
        cv2.imshow('Video', frame)

       # Taper sur la touche 'q' pour quitter
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
activation_reconnaissance_image()    

# Libération de la capture 
video_capture.release()
cv2.destroyAllWindows()

Ce qui donne les résultats suivants:

<table><td><img src = "23_Blog.png" width = "400px" ></td><td><img src = "31_Blog.png" width = "400px" ></td></table>


L'algorithme arrive bien à **me détecter**!

Pour être tout à fait honnete, il faudrait demander à **E.Macron** et **E.Philippe** de tester mon algorithme et ensuite affirmer que mon algorithme est fonctionnel. Comme je manque de temps (mais pas forcement d'ambition), je vous invite à le vérifier par vous même à l'aide du code ci-dessous.

#### Algorithme finale

In [ ]:
############################################################################################
#####                   Algorithme de reconnaissance faciale
#############################################################################################
##### Imporation des librairies
############################################################################################
import face_recognition
import cv2
import numpy as np

#### Lancement de la webcam
video_capture = cv2.VideoCapture(0)

##############################################################################################
#### Fonction de chargment de l'image à reconnaitre 
##############################################################################################

def chargement_image(fichier):
    """
        Entrée:
            Le fichier image
            
        Sortie:
            L'image et son encodage
    """
    #### On initialise les tableaux pour les objets "image" et "image_encod"
    image = np.array([])
    image_encod = np.array([])
    image = face_recognition.load_image_file(fichier)
    image_encod = face_recognition.face_encodings(image)[0]
    return image, image_encod #### Sortie: Image et Encodage de l'image

#### On crée trois nouvelles variables correspondant aux trois visages que l'on souhaite tester.

macron_img, obama_encod = chargement_image("E_macron.png")
philippe_img, biden_encod = chargement_image("E_Philippe.png")
moi_img, moi_encod = chargement_image("Moi.png")

####################################################################################################
#### Création de liste d'encoding et de label
####################################################################################################

visage_connu_encoding = [
    moi_encod, 
    obama_encod,
    biden_encod
]

nom_visage = [ "Yacine Aslimi",
                "Emmanuel Macron",
                "Edouard Philippe"
             ]


In [ ]:
#####################################################################################################
#####                       Comparaison des images avec le flux de la webcam
#####################################################################################################

def activation_reconnaissance_image():
    """
    But: Lancer la webcam et identifier si l'image correspond à une des images définie précédement
        Entrée:
            Aucune
        Sortie:
            Video et filtre sur le visage
    """
    
    while True:
       
        # On lance la webcam , le ret correspond et le frame correspond à l'image 
        ret, frame = video_capture.read()
        
        #### On convertie les images BGR en RGB
        rgb_frame = frame[:, :, ::-1]

        
        #### Détection de tout les visages sur le frame définie à l'aide de la fonction face_location
        face_locations = face_recognition.face_locations(rgb_frame)
        #### On encode ces visages
        face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

         #### On boucle sur chacun des frames de la video
        
        for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):

            #### Comparaison entre le visage connu et le frame  
            matches = face_recognition.compare_faces(visage_connu_encoding, face_encoding)#### False, False, False...True...False
            """ Dans cette fonction, on compare les visages encodées issues de nos images avec les visages de chacuns des frames issues de notre video"""
            name = "Inconnu"

            #### Si il ya correspondance entre visage_connu_encoding et les visages issues su frame (visages_encod)

            if True in matches:###( liste de true false (ici 3) et on demande s'il ya un true, on prend son index et on applique à la liste des nom de visage pour avoir le nom de l'image)
                first_match_index = matches.index(True) ### Numero de l'index (présence dans la liste)
                name = nom_visage[first_match_index] #### Application à la liste des nom de visage (index 3)

            # Déssine un rectangle sur le frame ou l'image est présente
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)

            # Rédaction d'un texte sous le cadre
            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 255, 0), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)


        #### Affiche le résultat de la video
        cv2.imshow('Video', frame)

       # Taper sur la touche 'q' pour quitter
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
activation_reconnaissance_image()    

# Libération de la capture 
video_capture.release()
cv2.destroyAllWindows()